#  Acquistion, agrégation et structuration de données

**Ce projet est à faire en groupe**


- G1: Mathieu, Adrien, Elliot
- G2: Yamine, Chris, Ahmed, Alan
- G3: Josephe, Arnaud, Guillaume 
- G4: Sébastien, Yann, Pierre, Valentin
- G5: Alexis, Thomas, Jules
- G6: 
- G7: 

Les personnes absentes le 07/11, contactez-moi.

## Consignes

Le but de cet exercice est de construire un conteneur de données qui contiendra des data sets concernant les **50 films les mieux notés selon [IMDB](https://www.imdb.com/list/ls055386972/) de 2001 à maintentant**.

**Attention** : *Avant de commencer, je vous invite à lire toutes les questions afin d'anticiper les informations qui vous seront demandées et mieux planifier vos traitements*

## Question 1 

### Extraction de données

Commencez par extraire les données dont vous aurez besoin pour répondre à la question posée. Vous pourrez combiner différentes méthodes pour extraires toutes les sources dont vous aurez besoin, comme par exemple:
- télécharger des data sets accessibles en Open Data à partir de [kaggle](https://www.kaggle.com/datasets?search=imbd)
- scraper la [page des 250 films les mieux notés d'IMDB](https://www.imdb.com/list/ls055386972/)
- utiliser des API(s) existantes pour récupérer des informations manquantes [API](https://sites.google.com/magicmakers.fr/teen-python/projets-dexploration/services-web/apis/api-imdb)

### Réponse 1

Affichez dans les cellules ci-dessous les structures de données dans lesquelles vous avez récupéré les informations pertinentes et donnez quelques caractéristiques de chacun de ce(s) data set(s) (par exemple taille, valeurs uniques, min , max ...)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:126.0) Gecko/20100101 Firefox/126.0",
    "Accept-Language":"en-US"
}

class imdbApi:

    def __init__(self) -> None:
        self.__key = "c183e392"
        self.__link = "http://www.omdbapi.com/?apikey="
        pass

    def getFilm(self,name)->list:
        requestApi = "http://www.omdbapi.com/?apikey=" + self.__key+"&t=" + name
        response = requests.get(requestApi)
        film = response.json()
        return film
    
        
class People :
    def __init__(self, name:str, work:str):
        self.__name = name
        self.work = work

    def setName(self, name:str):
        self.__name=name
    
    def getName(self):
        return self.__name

class Film : 
    def __init__(self,title:str) -> None:
        self.__title = title
        self.__img = ""
        self.__year =None
        self.__duration = None
        self.__genre = ""
        self.__actors = []
        self.__director = None
        self.__plot=""
        pass

    def get_image(self):
        return self.__img
    
    def set_image(self, src:str):
        self.__img=src
    
    def get_title(self):
        return self.__title
    
    def get_year(self):
        return self.__year
    
    def get_dureation(self):
        return self.__duration
    
    def set_year(self,year):
        self.__year = int(year)
    
    def set_duration(self, duration):
        self.__duration=duration

    def set_genre(self, genre:str):
        self.__genre = genre
    
    def get_genre(self):
        return self.__genre
    
    def add_actors(self, actor:People):
        self.__actors.append(actor)
    
    def get_actors(self):
        return self.__actors
    
    def set_director(self, director:People):
        self.__director = director
    
    def get_actors(self):
        return self.__director
    
    def set_plot(self, plot):
        self.__plot = plot

    def get_plot(self):
        return self.__plot
    
    def to_dict(self):
        names=[]
        for a in self.__actors:
            names.append(a.getName())
        if self.__director is None:
            director =""
        else:
            director = self.__director.getName()
        return {
            "Image": self.__img,
            "Titre": self.__title,
            "Genre": self.__genre,
            "Année": self.__year,
            "Duration": self.__duration,
            "Description":self.__plot,
            "Réalisateur": director,
            "Acteur": json.dumps(names)
        }



## Question 2 


### Filtrage et structuration des données

Vous allez maintenant filtrer vos données pour vous assurer qu'elles correspondent aux critères fixés dans les consignes et les structures en 3 dataframes distincts décrits ci dessous.

Vous présenterez les variables que vous avez retenues et donnerez un aperçu de leur quantité de valeurs manquantes

#### Dataframe contenant des informations spécifiques au film : 

Ce dataframe doit contenir à minima une colonne renseignant:
 - le titre du film
 - l'année de sa sortie en salle
 - la durée du film (runtime)
 - son genre
 - le nom de son réalisateur (director)
 - la liste des acteurs principaux

Et en bonus d'autres informations comme: 
 - une description du film
 - une photo de l'affiche du film

#### Dataframe contenant des informations concernant la popularité du film : 

Ce dataframe doit contenir à minima une colonne renseignant:
 - le nombre de votes d'utilisateurs pour ce film (échelle de 1 à 10)
 - la note moyenne des votes reçus (échelle de 1 à 10)
 - les recettes du film (en millions de $) 
 
Et en bonus d'autres informations comme: 
- la note mediane des votes reçus (échelle de 1 à 10)
- l'écart type des votes reçus

#### Dataframe contenant des informations concernant chaque acteur et réalisateur présents dans ces films : 

Ce dataframe doit contenir à minima une colonne renseignant:
- leur nom 
- leur pays de naissance
- leur age
- s'il est toujours vivant
- son/ses métier(s) (a choisir parmi la liste: acteur, realisateur, producteur)

Et en bonus d'autres informations comme: 
- une photo

### Réponse 2

Afficher dans les cellules ci dessous les informations concernant les dataframes 

In [126]:
import sys

class Popularity:
    def __init__(self,film:Film, numberVote, rating:float,income:str) -> None:
        if not  income is None and income != "N/A":
            value = income.split("$")[1].replace(",","")
        else:
            value = 0.00
        self.__film = film
        self.__nbVote = convertStringVote(numberVote)
        self.__rating = round(float(rating),2)
        self.__income = round(float(value),2)
        pass


    ##########################################################
    ######## Calcul de la median face au vote maximal ########
    ##########################################################
    def __median(self, max:int)->float:
        return round(self.__nbVote / max *10,2)

    ##########################################################
    ################# Calcul de l'écart type #################
    ##########################################################
    def __ecartType(self,median)->float:
        return abs(median-float(self.__rating))
    
    def __max(self,listVote):
        max = 0
        for e in listVote : 
            if e> max : 
                max = e
        return max
    
        
    def set_income(self, income:str):
        self.__income = round(income.split("$")[1].replace(",",""),2)

    def get_income(self)->float:
        return self.__income   
    
    def to_dict(self, listVote):
        max = self.__max(listVote)
        median = self.__median(max)
        ecartType = self.__ecartType(median=median)
        return {
            "Film" : self.__film.get_title(),
            "Recette" : self.__income,
            "nombre de vote": self.__nbVote,
            "note moyenne": self.__rating,
            "Note Medianne": median,
            "Ecart Type": ecartType
        }
    
    def getVote(self)->int:
        return self.__nbVote


def getVotes():
    listVote = []
    for p in popularities:
        listVote.append(p.getVote())
    return listVote

def convertStringVote( vote:str)->int:
    vote = vote.replace(",","")
    return int(vote)
            
        


## Question 3 

### Nettoyage des données

Realisez des opérations de nettoyage des données afin de rendre vos data sets le plus facilement exploitables. Vous réaliserez **a minima** les traitements suivants:
- convertir les différents types de données rencontrées dans un format adapté pour faciliter les traitements futurs et gagner en RAM
- vérification et/ou traitement des valeurs manquantes,  des doublons (par exemple des noms identiques, ou écrits avec des orthographes différentes) 
- vérification et traitement des outliers

### Réponse 3

## Question 4 

### Agrégation et structuration des données

Vos données étant nettoyées, vous allez maintenant les structurez et les stocker dans un **conteneur de données au format HDF5**. Vous êtes libre d'architecturer votre conteneur comme vous le souhaitez créerez a minima deux groupes:
- un groupe contenant les images que vous allez acquérir
- un groupe pour chaque dataset

### Réponse 4

Affichez à minima, des informations concernant chaque groupe ainsi que les data sets qu'ils contient

In [134]:
import h5py
import numpy as np

with h5py.File('data.hdf5', 'w') as f:
    for col in df.columns:
        f.create_dataset("Cinema/"+col, data=df[col].values)
    for col in df_popularity:
        f.create_dataset("Film/"+col, data=df_popularity[col].values)


## Question 5 

### Exploitation de vos données

[tutorial HDF5](https://deusyss.developpez.com/tutoriels/Python/hdf5/)

A partir de votre conteneur de données HDF5 vous allez extraire, filtrer et agréger vos données pour répondre aux questions suivantes:
- Quelle est la durée médiane des films ?
- Quels sont les notes moyennes des films par genre ? 
- Faites un graphique pour représenter la répartition des durées des films 
- Quels sont les caractéristiques des films (genre, durée, ...) et acteurs (vivant ou décédé, age, métier, ...),  classés dans les 5% des films les mieux notés ? 

### Réponse 5

# Code principal

In [128]:
response = requests.get('https://www.imdb.com/chart/top/', headers=headers).text
soup = BeautifulSoup(response, 'html.parser')
list = soup.find_all(class_="cli-parent")
films=[]
peoples = []
popularities = []
apiWork = imdbApi()
for l in list:
    film = Film(l.find(class_="ipc-title__text").text.split(" ",1)[1])
    film.set_image(l.find(class_="ipc-image")["src"])
    film_information=l.find(class_="cli-title-metadata").contents
    film.set_year(film_information[0].text)
    film.set_duration(film_information[1].text)
    apiFilm = apiWork.getFilm(name = film.get_title())
    film.set_genre(apiFilm["Genre"])
    film.set_plot(apiFilm["Plot"])
    director = People(apiFilm["Director"],"Réalisateur")
    film.set_director(director)
    if director not in peoples:
        peoples.append(director)
    actors = apiFilm["Actors"].split(", ")
    for a in actors:
        actor = People(a,"Acteur")
        film.add_actors(actor=actor)
        if actor not in peoples:
            peoples.append(actor)
    popularity = Popularity(film=film, numberVote= apiFilm["imdbVotes"], rating=apiFilm["imdbRating"],income=apiFilm["BoxOffice"])
    popularities.append(popularity)
    films.append(film)

df = pd.DataFrame(f.to_dict() for f in films)
df_popularity = pd.DataFrame(p.to_dict(listVote = getVotes()) for p in popularities)



In [129]:
df

,Image,Titre,Genre,Année,Duration,Description,Réalisateur,Acteur
0,https://m.media-amazon.com/images/M/MV5BMDAyY2...,The Shawshank Redemption,Drama,1994,2h 22m,A banker convicted of uxoricide forms a friend...,Frank Darabont,"[""Tim Robbins"", ""Morgan Freeman"", ""Bob Gunton""]"
1,https://m.media-amazon.com/images/M/MV5BYTJkNG...,The Godfather,"Crime, Drama",1972,2h 55m,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"[""Marlon Brando"", ""Al Pacino"", ""James Caan""]"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,"Action, Crime, Drama",2008,2h 32m,When a menace known as the Joker wreaks havoc ...,Christopher Nolan,"[""Christian Bale"", ""Heath Ledger"", ""Aaron Eckh..."
3,https://m.media-amazon.com/images/M/MV5BNzc1OW...,The Godfather Part II,"Crime, Drama",1974,3h 22m,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"[""Al Pacino"", ""Robert De Niro"", ""Robert Duvall""]"
4,https://m.media-amazon.com/images/M/MV5BYjE4Nz...,12 Angry Men,"Crime, Drama",1957,1h 36m,The jury in a New York City murder trial is fr...,Sidney Lumet,"[""Henry Fonda"", ""Lee J. Cobb"", ""Martin Balsam""]"
5,https://m.media-amazon.com/images/M/MV5BMTZkMj...,The Lord of the Rings: The Return of the King,"Action, Adventure, Drama",2003,3h 21m,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"[""Elijah Wood"", ""Viggo Mortensen"", ""Ian McKell..."
6,https://m.media-amazon.com/images/M/MV5BNjM1ZD...,Schindler's List,"Biography, Drama, History",1993,3h 15m,"In German-occupied Poland during World War II,...",Steven Spielberg,"[""Liam Neeson"", ""Ralph Fiennes"", ""Ben Kingsley""]"
7,https://m.media-amazon.com/images/M/MV5BYTViYT...,Pulp Fiction,"Crime, Drama",1994,2h 34m,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"[""John Travolta"", ""Uma Thurman"", ""Samuel L. Ja..."
8,https://m.media-amazon.com/images/M/MV5BNzIxMD...,The Lord of the Rings: The Fellowship of the Ring,"Action, Adventure, Drama",2001,2h 58m,A meek Hobbit from the Shire and eight compani...,Peter Jackson,"[""Elijah Wood"", ""Ian McKellen"", ""Orlando Bloom""]"
9,https://m.media-amazon.com/images/M/MV5BMWM5Zj...,"The Good, the Bad and the Ugly","Adventure, Drama, Western",1966,2h 58m,A bounty hunting scam joins two men in an unea...,Sergio Leone,"[""Clint Eastwood"", ""Eli Wallach"", ""Lee Van Cle..."


In [130]:
df_popularity

,Film,Recette,nombre de vote,note moyenne,Note Medianne,Ecart Type
0,The Shawshank Redemption,28767189.0,2957050,9.3,10.00,0.70
1,The Godfather,136381073.0,2057473,9.2,6.96,2.24
2,The Dark Knight,534987076.0,2932729,9.0,9.92,0.92
3,The Godfather Part II,47834595.0,1392629,9.0,4.71,4.29
4,12 Angry Men,0.0,886741,9.0,3.00,6.00
5,The Lord of the Rings: The Return of the King,381878219.0,2024305,9.0,6.85,2.15
6,Schindler's List,96898818.0,1480596,9.0,5.01,3.99
7,Pulp Fiction,107928762.0,2266472,8.9,7.66,1.24
8,The Lord of the Rings: The Fellowship of the Ring,319372078.0,2054361,8.9,6.95,1.95
9,"The Good, the Bad and the Ugly",25100000.0,828737,8.8,2.80,6.00
